# Dataset

In [61]:
import pandas as pd

In [62]:
# Load the cleaned datasets
netflix_df = pd.read_csv('https://raw.githubusercontent.com/anthonybrown0528/csc-442-course-project/refs/heads/main/dataset/clean/netflix_film_data.csv')
imdb_df = pd.read_csv('https://raw.githubusercontent.com/anthonybrown0528/csc-442-course-project/refs/heads/main/dataset/clean/netflix_imdb_scores.csv')

## 1. Data Merging

### 1.1 Identify Common Columns

In [63]:
netflix_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7293 entries, 0 to 7292
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   show_id          7293 non-null   object 
 1   type             7293 non-null   object 
 2   title            7293 non-null   object 
 3   director         7293 non-null   object 
 4   cast             7293 non-null   object 
 5   country          7293 non-null   object 
 6   date_added       7293 non-null   object 
 7   release_year     7293 non-null   int64  
 8   rating           7290 non-null   object 
 9   num_releases     7293 non-null   int64  
 10  listed_in        7293 non-null   object 
 11  description      7293 non-null   object 
 12  num_listed_in    7293 non-null   int64  
 13  first_cast       7293 non-null   object 
 14  cast_freq_mean   7293 non-null   float64
 15  genre_freq_mean  7293 non-null   float64
dtypes: float64(2), int64(3), object(11)
memory usage: 911.8+ KB


In [64]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5278 entries, 0 to 5277
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              5278 non-null   object 
 1   type               5278 non-null   object 
 2   description        5278 non-null   object 
 3   release_year       5278 non-null   int64  
 4   age_certification  2996 non-null   object 
 5   runtime            5278 non-null   int64  
 6   imdb_id            5278 non-null   object 
 7   imdb_score         5278 non-null   float64
 8   imdb_votes         5278 non-null   int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 371.2+ KB


In [65]:
common_titles = list(set(imdb_df["title"]) & set(netflix_df["title"]))
print('Common Titles:', len(common_titles))

common_release_year = list(set(imdb_df["release_year"]) & set(netflix_df["release_year"]))
print('Common Release Years:', len(common_release_year))

common_age_certification_year = list(set(imdb_df["age_certification"]) & set(netflix_df["rating"]))
print('Common Age Certifications:', len(common_age_certification_year))

Common Titles: 3177
Common Release Years: 64
Common Age Certifications: 12


The datasets share the `title`, `release_year`, and `age_certification` columns and contain common values. The title and release year are sufficient to identify a unique film in this dataset.

## 1.2 Broad Merge

In [66]:
# Rename the rating column to age_certification to be consistent with the
# column label in the other dataset
netflix_df = netflix_df.rename(columns={'rating': 'age_certification'})

In [67]:
netflix_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,age_certification,num_releases,listed_in,description,num_listed_in,first_cast,cast_freq_mean,genre_freq_mean
0,s2,TV Show,Blood & Water,Unknown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",3,Ama Qamata,1.157895,597.666667
1,s5,TV Show,Kota Factory,Unknown,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,3,Mayur More,2.000000,608.000000
2,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,1,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s...",3,Kofi Ghanaba,1.000000,1814.666667
3,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...,2,Mel Giedroyc,2.000000,171.000000
4,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,1,"Comedies, Dramas",A woman adjusting to life after a loss contend...,2,Melissa McCarthy,5.090909,1942.000000


In [68]:
# Perform an inner join on film title and release year
netflix_film_imdb_data = pd.merge(imdb_df, netflix_df, how='inner', on=['title', 'release_year'])

# The duplicate type and age certification are redundant
# Remove one of each from the merged data and retain the columns which
# do not contain missing values
#
# Merging with duplicate columns forced a renaming,
# so revert the columns back to the original naming
netflix_film_imdb_data = netflix_film_imdb_data \
            .drop(columns=['type_x', 'age_certification_x']) \
            .rename(columns={'type_y': 'type', 'age_certification_y': 'age_certification'})

netflix_film_imdb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2477 entries, 0 to 2476
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              2477 non-null   object 
 1   description_x      2477 non-null   object 
 2   release_year       2477 non-null   int64  
 3   runtime            2477 non-null   int64  
 4   imdb_id            2477 non-null   object 
 5   imdb_score         2477 non-null   float64
 6   imdb_votes         2477 non-null   int64  
 7   show_id            2477 non-null   object 
 8   type               2477 non-null   object 
 9   director           2477 non-null   object 
 10  cast               2477 non-null   object 
 11  country            2477 non-null   object 
 12  date_added         2477 non-null   object 
 13  age_certification  2477 non-null   object 
 14  num_releases       2477 non-null   int64  
 15  listed_in          2477 non-null   object 
 16  description_y      2477 

In [69]:
netflix_film_imdb_data.to_csv('netflix_film_imdb_data.csv', index=False)

# Credit

This notebook contains contributions from Anthony Brown.

The organization of this notebook is inspired by a notebook provided as a workshop for CSC 442 at NC State University. This workshop was created by Aditi Mallavarapu, Claire Cahoon and Walt Gurley, adapted from previous workshop materials by Scott Bailey and Simon Wiles, of Stanford Libraries.